In [11]:
import tensorflow as tf 

tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [12]:

import tensorflow as tf
import cv2
import numpy as np
from PIL import Image
import os
import time

from configuration import Config
from core.efficientdet import EfficientDet, PostProcessing
from data.dataloader import DataLoader

In [13]:
# 원래 모델

def idx2class():
    return dict((v, k) for k, v in Config.pascal_voc_classes.items())


def draw_boxes_on_image(image, boxes, scores, classes):
    num_boxes = boxes.shape[0]
    for i in range(num_boxes):
        class_and_score = str(idx2class()[classes[i]]) + ": " + str(scores[i])
        cv2.rectangle(img=image, pt1=(boxes[i, 0], boxes[i, 1]), pt2=(boxes[i, 2], boxes[i, 3]), color=(255, 0, 0), thickness=2)
        cv2.putText(img=image, text=class_and_score, org=(boxes[i, 0], boxes[i, 1] - 10), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1.5, color=(0, 255, 255), thickness=2)
    return image

def test_single_picture(picture_dir, model):
    image_array = cv2.imread(picture_dir)
    print(image_array.shape)
    image = DataLoader.image_preprocess(is_training=False, image_dir=picture_dir)
    image = tf.expand_dims(input=image, axis=0)

    outputs = model(image, training=True)
    post_process = PostProcessing()
    boxes, scores, classes = post_process.testing_procedure(outputs, [image_array.shape[0], image_array.shape[1]])
#     print("-"*50)
#     print("boxes")
#     print(boxes)
    print("-" * 50)
    print("scores")
    print(scores)
#     print("-" * 50)
#     print("classes")
#     print(classes)
#     print("-" * 50)
    image_with_boxes = draw_boxes_on_image(image_array, boxes.astype(np.int), scores, classes)
    return image_with_boxes

# def test_batch_picture(image_list, model):
#     outputs = model(image_list_batch, training=True)
#     post_process = PostProcessing()
#     boxes = []
#     scores = []
#     classes = []
    
#     for i in range(32):
#         image_array  = image_list[i]
#         image  = tf.expand_dims(input=image_array, axis=0)     
#         print(outputs[i].shape)
#         print(image_list[i].shape)
#         print( [ image_array.shape[0], image_array.shape[1]])
#         boxes[i], scores[i], classes[i] = post_process.testing_procedure(outputs[i], [ image_array.shape[0], image_array.shape[1]])
    
#     print("boxes : ", boxes.shape)
#     print("scores : ", scores.shape)
#     print("classes : ", classes.shape)
#     image_with_boxes = draw_boxes_on_image(image_array, boxes.astype(np.int), scores, classes)
#     return image_with_boxes

# def data_load(image_path):
#     path = image_path
#     count = 0
#     image_list = []
#     for item in os.listdir(path)[:32]:
#         imgpath = path +'/' + item
#         img = Image.open(imgpath)
#         arr = np.array(img)
#         image_list.append(arr)
#         #img.save(str(count) +  "output.png")
#         count = count +1
#     image_batch =  tf.image.resize(image_list, (512,512))
# #     image_temp = tf.data.Dataset.from_tensor_slices(image_list).batch(32).take(1)
# #     image_batch =  tf.image.resize(image_temp, (512,512))
#     return image_batch

In [14]:
if __name__ == '__main__':
    # GPU settings
    gpus = tf.config.list_physical_devices("GPU")
    if gpus:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            
            
    #load_weights_from_epoch = Config.load_weights_from_epoch_quan
    
    efficientdet = EfficientDet()
    #efficientdet.load_weights(filepath=Config.save_model_dir + "saved_model")
 

    load_weights_from_epoch = 100
    efficientdet.load_weights(filepath=Config.best_model_dir+"epoch-{}".format(load_weights_from_epoch))
    #efficientdet.load_weights(filepath=Config.best_model_dir+"epoch-{}_cus".format(load_weights_from_epoch))
    #efficientdet.load_weights(filepath=Config.save_model_dir+"epoch-{}".format(load_weights_from_epoch))
    #efficientdet.load_weights(filepath=Config.save_model_dir+"epoch-{}".format(load_weights_from_epoch))
    
#     image_path = './data/fire_smoke/train/JPEGImages'
#     image_list_batch =  data_load(image_path)
    
#     t1 = time.time()
#     outputs = efficientdet(image_list_batch, training=True)
#     t2 = time.time()
#     print("------------------------------------")
#     print(t2-t1)
#     print(outputs.shape)
    
    
#     image_output_list = test_batch_picture(image_list_batch, efficientdet)
#     print(image_output_list.shape)
    
    test_image_dir_1 = "./test_pictures/KakaoTalk_20211103_192433256.png"

#     test_image_dir_2 = "./test_pictures/ck0kewsaha6hh07215jgx1bp2_jpeg.rf.1a375d20560d0de016bb524921f7b2a9.jpg"
    
#     test_image_dir_3 = "./test_pictures/ck0kfhu4n8q7f0701ixmonyig_jpeg.rf.a3cc5282520b3bac90718bdd5528bd76.jpg"

#     test_image_dir_4 = "./test_pictures/smoking_women.jpg"
    
    
    image = test_single_picture(picture_dir=test_image_dir_1, model=efficientdet)
    cv2.namedWindow("detect result", flags=cv2.WINDOW_NORMAL)
    cv2.imshow("detect result", image)
    cv2.waitKey(0)
    
#     image = test_single_picture(picture_dir=test_image_dir_2, model=efficientdet)
#     cv2.namedWindow("detect result", flags=cv2.WINDOW_NORMAL)
#     cv2.imshow("detect result", image)
#     cv2.waitKey(0)
    
#     image = test_single_picture(picture_dir=test_image_dir_3, model=efficientdet)
#     cv2.namedWindow("detect result", flags=cv2.WINDOW_NORMAL)
#     cv2.imshow("detect result", image)
#     cv2.waitKey(0)
    
#     image = test_single_picture(picture_dir=test_image_dir_4, model=efficientdet)
#     print(image.shape)
#     cv2.namedWindow("detect result", flags=cv2.WINDOW_NORMAL)
#     cv2.imshow("detect result", image)
#     cv2.waitKey(0)

AttributeError: type object 'Config' has no attribute 'best_model_dir'

In [5]:
cv2.imshow("detect result", image)
cv2.waitKey(0)

-1

In [ ]:
# tflite 평가 하기

def idx2class():
    return dict((v, k) for k, v in Config.pascal_voc_classes.items())


def draw_boxes_on_image(image, boxes, scores, classes):
    num_boxes = boxes.shape[0]
    for i in range(num_boxes):
        class_and_score = str(idx2class()[classes[i]]) + ": " + str(scores[i])
        cv2.rectangle(img=image, pt1=(boxes[i, 0], boxes[i, 1]), pt2=(boxes[i, 2], boxes[i, 3]), color=(255, 0, 0), thickness=2)
        cv2.putText(img=image, text=class_and_score, org=(boxes[i, 0], boxes[i, 1] - 10), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1.5, color=(0, 255, 255), thickness=2)
    return image

def load_model(path):  
    interpreter = tf.lite.Interpreter(model_path = path)
    
    interpreter.resize_tensor_input(0, [ 1 , 640, 640,   3])
#     interpreter.resize_tensor_input(856, [ 1, 49104,     5])
    
    interpreter.allocate_tensors()
    return interpreter

def test_single_picture(picture_dir, model):
    image_array = cv2.imread(picture_dir)
    image = DataLoader.image_preprocess(is_training=False, image_dir=picture_dir)
    image = tf.expand_dims(input=image, axis=0)
    print(image.shape)
    
    input_data = np.array(image, dtype = np.uint8)
    input_details = model.get_input_details()[0]
    output_details = model.get_output_details()[0]
    model.set_tensor(input_details['index'], input_data)
    model.invoke()
    outputs= model.get_tensor(output_details['index'])
    
    print(outputs)
    # 원래 outputs =  소수점 형태
    # quantizaation model =  숫자 너무큼.. output_tensor shape 해결하기
    
    outputs = tf.constant(outputs)
    post_process = PostProcessing()
    boxes, scores, classes = post_process.testing_procedure(outputs, [image_array.shape[0], image_array.shape[1]])
    print("-"*50)
    print("boxes")
    print(boxes)
    print("-" * 50)
    print("scores")
    print(scores)
    print("-" * 50)
    print("classes")
    print(classes)
    print("-" * 50)
    image_with_boxes = draw_boxes_on_image(image_array, boxes.astype(np.int), scores, classes)
    return image_with_boxes


2021-10-30 17:27:13.074276: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-30 17:27:13.794591: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22854 MB memory:  -> device: 0, name: NVIDIA TITAN RTX, pci bus id: 0000:18:00.0, compute capability: 7.5


(1, 640, 640, 3)


In [ ]:
def writeVideo():

    video_capture = cv2.VideoCapture('rtsp://----------')
    
    video_capture.set(3, 800)  # 영상 가로길이 설정
    video_capture.set(4, 600)  # 영상 세로길이 설정
    fps = 20
    # 가로 길이 가져오기
    streaming_window_width = int(video_capture.get(3))
    # 세로 길이 가져오기
    streaming_window_height = int(video_capture.get(4))  

    #파일 저장하기 위한 변수 선언
    path = f'D:/cctv/cctv/python/{fileName}.avi'
    fourcc = cv2.VideoWriter_fourcc('X', 'V', 'I', 'D')
    

In [ ]:
gpus = tf.config.list_physical_devices("GPU")

if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True
                                                 
model_path = "./tflite_model_quant_D1_SGD.tflite"
efficientdet_lite = load_model(model_path)   
                                                 
cap = cv2.VideoCapture(args.camera_idx)
                                                 
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    cv2_im = frame

    cv2_im_rgb = cv2.cvtColor(cv2_im, cv2.COLOR_BGR2RGB)
    cv2_im_rgb = cv2.resize(cv2_im_rgb, inference_size)
    cv2_im = test_single_picture(interpreter, cv2_im_rgb.tobytes())
    cv2.imshow('frame', cv2_im)
    if cv2.waitKey(1) & 0xFF == ord('q'):
         break

cap.release()
cv2.destroyAllWindows()                                        
                                                 

In [ ]:

gpus = tf.config.list_physical_devices("GPU")

if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

model_path = "./tflite_model_quant_D1_SGD.tflite"
efficientdet_lite = load_model(model_path)

test_image_dir_1 = "./test_pictures/ck0kepbs9kdym0848hgpcf3y9_jpeg.rf.d0a63becb54a83b6b026f4b38a42933b.jpg"

test_image_dir_2 = "./test_pictures/ck0kewsaha6hh07215jgx1bp2_jpeg.rf.1a375d20560d0de016bb524921f7b2a9.jpg"
    
test_image_dir_3 = "./test_pictures/ck0kfhu4n8q7f0701ixmonyig_jpeg.rf.a3cc5282520b3bac90718bdd5528bd76.jpg"

test_image_dir_4 = "./test_pictures/smoking_women.jpg"
    
image = test_single_picture(picture_dir=test_image_dir_1, model=efficientdet_lite)
cv2.namedWindow("detect result", flags=cv2.WINDOW_NORMAL)
cv2.imshow("detect result", image)
cv2.waitKey(0)
    
image = test_single_picture(picture_dir=test_image_dir_2, model=efficientdet_lite)
cv2.namedWindow("detect result", flags=cv2.WINDOW_NORMAL)
cv2.imshow("detect result", image)
cv2.waitKey(0)
    
image = test_single_picture(picture_dir=test_image_dir_3, model=efficientdet_lite)
cv2.namedWindow("detect result", flags=cv2.WINDOW_NORMAL)
cv2.imshow("detect result", image)
cv2.waitKey(0)
    
image = test_single_picture(picture_dir=test_image_dir_4, model=efficientdet_lite)
cv2.namedWindow("detect result", flags=cv2.WINDOW_NORMAL)
cv2.imshow("detect result", image)
cv2.waitKey(0)

In [5]:
!nvidia-smi

Sun Nov  7 21:33:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN RTX    Off  | 00000000:86:00.0 Off |                  N/A |
| 41%   42C    P8    21W / 280W |   1116MiB / 24220MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------